In [ ]:
# pip install mediapipe opencv-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 15.8 MB/s eta 0:00:00


In [ ]:
!pip install mediapipe

In [9]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Initialize video capture
cap = cv2.VideoCapture('/content/dl_video.mp4')  # Replace with the path to your video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create an output video writer
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output_video_final.avi', fourcc, 20.0, (frame_width, frame_height))
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = pose.process(frame_rgb)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        pose_keypoints = [(int(l.x * frame_width), int(l.y * frame_height)) for l in landmarks]
        # TASK1-CREATING BOUNDING BOX TO IDENTIFY PERSONS
        bounding_box = cv2.boundingRect(np.array(pose_keypoints))
        frame = cv2.rectangle(frame, (bounding_box[0], bounding_box[1]), (bounding_box[0] + bounding_box[2], bounding_box[1] + bounding_box[3]), (0, 255, 0), 2)
        # Creating pose keypoints
        # Task 2-pose estimation
        for landmark in landmarks:
            x, y = int(landmark.x * frame_width), int(landmark.y * frame_height)
            frame = cv2.circle(frame, (x, y), 4, (0, 0, 255), -1)
        out.write(frame)
# TASK3-Recognizing actions of a person
action_classes = {
    "Standing": [mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER],
    "Walking": [mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.RIGHT_HIP],
    "Running": [mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.RIGHT_KNEE],
    "Shoplifting": [mp_pose.PoseLandmark.RIGHT_WRIST, mp_pose.PoseLandmark.LEFT_WRIST],
}
cap1=cv2.VideoCapture('/content/output_video_final.avi')
frame_width1 = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height1 = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))
out1 = cv2.VideoWriter('output_video_final1.avi', fourcc, 20.0, (frame_width1, frame_height1))

while cap1.isOpened():
    ret, frame = cap1.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = pose.process(frame_rgb)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        detected_actions = []
        for action, keypoints in action_classes.items():
            if all(landmarks[keypoint].visibility > 0.5 for keypoint in keypoints):
                detected_actions.append(action)

        if detected_actions:
            action = ", ".join(detected_actions)
        else:
            action = "Undefined"

        cv2.putText(frame, f'Action: {action}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    out1.write(frame)

cap1.release()
out1.release()
cv2.destroyAllWindows()